In [1]:
import keras
import tensorflow as tf
import datetime
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers

#DATA
from keras.datasets import boston_housing


In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
mean = train_data.mean(axis=0)
std = train_data.mean(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

In [9]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss = 'mse', metrics=['mae'])
    return model

In [13]:
#K-fold Validation
import numpy as np
 
k = 4
num_val_samples = len(train_data) // k

num_epochs = 100
scores = []

for i in range(k):
    val_data = train_data[num_val_samples*i:num_val_samples*(i+1)]
    val_targets = train_targets[num_val_samples*i:num_val_samples*(i+1)]

    partial_train_data = np.concatenate([train_data[: num_val_samples * i], train_data[num_val_samples * ( i + 1 ):]], axis = 0)
    partial_train_targets = np.concatenate([train_targets[:num_val_samples*i], train_targets[num_val_samples*(i+1):]], axis = 0)

   
    model = build_model()

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size=1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
    scores.append(val_mae)

print(np.mean(val_mae))

2.6633098125457764


In [12]:
%tensorboard --logdir logs/fit